In [121]:
!pip install -q streamlit requests gTTS pillow python-dotenv



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
import os, uuid, base64, json
from io import BytesIO
from pathlib import Path
from typing import List
import requests
from gtts import gTTS
from PIL import Image
from dotenv import load_dotenv

# Load .env if present
load_dotenv()

# ---- CONFIG ----
HF_TOKEN = os.getenv("HF_TOKEN", "")  # safer: set in Streamlit Cloud secrets
MODEL_TEXT = "HuggingFaceH4/zephyr-7b-beta"  # text generation model
MODEL_IMAGE = "stabilityai/stable-diffusion-xl-base-1.0"      # stil need fixing optional image model
N_FOLLOWUPS = 3
OUTPUT_DIR = Path("outputs"); OUTPUT_DIR.mkdir(exist_ok=True)


In [158]:
FOLLOWUP_PROMPT = """You are an insightful, concise dream analyst.
Ask {n} SHORT follow-up questions to better understand this dream. 
No preamble, just numbered questions on separate lines.

Dream:
\"\"\"{dream}\"\"\""""

POEM_PROMPT = """You are a poetic interpreter of dreams.
Using the dream and answers, write a flowing poetic prose (150–250 words). 
No bullet points, no headers—just a single evocative passage.

Context:
\"\"\"{context}\"\"\""""

IMAGE_PROMPT_TMPL = """Write a single <120 word prompt for an image generator capturing the symbolism, mood and key elements of this dream context:

\"\"\"{context}\"\"\""""


In [159]:
# ===== Cell 4 — Hugging Face helpers (patched with wait_for_model + clear errors) =====
import requests, base64
from typing import List

def _raise_for_hf_error(resp: requests.Response):
    """Give clearer messages than requests.raise_for_status()."""
    if resp.status_code == 401:
        raise RuntimeError("HF 401 Unauthorized: Token missing/invalid. Check HF_TOKEN.")
    if resp.status_code == 403:
        raise RuntimeError("HF 403 Forbidden: You don't have access to this model.")
    if resp.status_code == 429:
        raise RuntimeError("HF 429 Rate limit: Too many requests. Slow down or upgrade.")
    if resp.status_code == 503:
        # common when model is loading; wait_for_model=true should help, but still…
        raise RuntimeError("HF 503: Model loading or unavailable. Try again in a few seconds.")
    resp.raise_for_status()
    data = resp.json()
    if isinstance(data, dict) and "error" in data:
        raise RuntimeError(f'HF Error: {data["error"]}')
    return data

def hf_text(prompt: str,
            model: str = MODEL_TEXT,
            max_new_tokens: int = 300,
            temperature: float = 0.8) -> str:
    url = f"https://api-inference.huggingface.co/models/{model}?wait_for_model=true"
    headers = {"Authorization": f"Bearer {HF_TOKEN}"}
    payload = {
        "inputs": prompt,
        "parameters": {
            "max_new_tokens": max_new_tokens,
            "temperature": temperature
        }
    }
    r = requests.post(url, headers=headers, json=payload, timeout=180)
    data = _raise_for_hf_error(r)
    # HF returns a list of dicts: [{'generated_text': '...'}]
    return data[0]["generated_text"].strip()

def clean_questions(raw: str, n: int) -> List[str]:
    lines = [ln.strip() for ln in raw.splitlines() if ln.strip()]
    qs = []
    for ln in lines:
        ln = ln.lstrip("0123456789).:- ").strip()
        if ln:
            qs.append(ln)
    return qs[:n]

def gen_followups(dream: str, n: int = N_FOLLOWUPS) -> List[str]:
    raw = hf_text(FOLLOWUP_PROMPT.format(dream=dream, n=n))
    return clean_questions(raw, n)

def gen_poem(context: str) -> str:
    return hf_text(POEM_PROMPT.format(context=context),
                   temperature=0.85,
                   max_new_tokens=400)

def gen_image(context: str) -> bytes:
    # 1) Ask text model to craft a concise SDXL prompt
    img_prompt = hf_text(IMAGE_PROMPT_TMPL.format(context=context),
                         temperature=0.6,
                         max_new_tokens=120)
    # 2) Call SDXL (binary image response)
    url = f"https://api-inference.huggingface.co/models/{MODEL_IMAGE}?wait_for_model=true"
    headers = {"Authorization": f"Bearer {HF_TOKEN}"}
    payload = {"inputs": img_prompt}
    r = requests.post(url, headers=headers, json=payload, timeout=240)
    _raise_for_hf_error(r)  # will raise if JSON w/ error
    return r.content  # raw PNG/JPEG bytes


In [160]:
def make_tts_mp3(text: str) -> Path:
    path = OUTPUT_DIR / f"tts_{uuid.uuid4().hex}.mp3"
    gTTS(text).save(str(path))
    return path

def save_image_bytes(img_bytes: bytes) -> Path:
    path = OUTPUT_DIR / f"img_{uuid.uuid4().hex}.png"
    with open(path, "wb") as f:
        f.write(img_bytes)
    return path


In [161]:
def run_cli():
    dream = input("Describe your dream:\n> ")
    qs = gen_followups(dream)
    print("\nFollow-up questions:")
    answers = []
    for i, q in enumerate(qs, 1):
        ans = input(f"{i}) {q}\n> ")
        answers.append(ans)

    context = f"Dream: {dream}\nAnswers:\n" + "\n".join(f"- {a}" for a in answers)

    mode = input("\nOutput type? [text/image] default=text: ").strip().lower() or "text"

    if mode == "image":
        prose = gen_poem(context)
        img_bytes = gen_image(context)
        img_path = save_image_bytes(img_bytes)
        print("\n--- PROSE ---\n")
        print(prose)
        display(Image.open(img_path))
    else:
        prose = gen_poem(context)
        print("\n--- PROSE ---\n")
        print(prose)

    if input("\nMake TTS? [y/N]: ").strip().lower() == "y":
        audio_path = make_tts_mp3(prose)
        print(f"Saved audio to {audio_path}")

# Uncomment to test interactively:
# run_cli()


In [ ]:
#%%writefile streamlit_app.py
import os, uuid, base64
from io import BytesIO
from pathlib import Path
import requests
from gtts import gTTS
from PIL import Image
import streamlit as st

# ------------- CONFIG -------------
HF_TOKEN = os.getenv("HF_TOKEN", "")
MODEL_TEXT  = "HuggingFaceH4/zephyr-7b-beta"
MODEL_IMAGE = "stabilityai/stable-diffusion-xl-base-1.0"
N_FOLLOWUPS = 3
OUTPUT_DIR = Path("outputs"); OUTPUT_DIR.mkdir(exist_ok=True)

FOLLOWUP_PROMPT = """You are an insightful, concise dream analyst.
Ask {n} SHORT follow-up questions to better understand this dream.
No preamble, just numbered questions on separate lines.

Dream:
\"\"\"{dream}\"\"\""""

POEM_PROMPT = """You are a poetic interpreter of dreams.
Using the dream and answers, write a flowing poetic prose (150–250 words). 
No bullet points or headers—just a single evocative passage.

Context:
\"\"\"{context}\"\"\""""

IMAGE_PROMPT_TMPL = """Write a single <120 word prompt for an image generator capturing the symbolism, mood and key elements of this dream context:

\"\"\"{context}\"\"\""""

from openai import OpenAI

# Set up OpenAI-style Hugging Face router client
client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=HF_TOKEN,
)

def hf_text(prompt: str,
            model: str = "HuggingFaceH4/zephyr-7b-beta:featherless-ai",
            max_new_tokens: int = 300,
            temperature: float = 0.8) -> str:
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": prompt}
        ],
        max_tokens=max_new_tokens,
        temperature=temperature,
    )
    return response.choices[0].message.content.strip()


def clean_questions(raw: str, n: int):
    lines = [ln.strip() for ln in raw.splitlines() if ln.strip()]
    qs = []
    for ln in lines:
        ln = ln.lstrip("0123456789).:- ").strip()
        if ln:
            qs.append(ln)
    return qs[:n]

def gen_followups(dream: str, n: int = N_FOLLOWUPS):
    raw = hf_text(FOLLOWUP_PROMPT.format(dream=dream, n=n))
    return clean_questions(raw, n)

def gen_poem(context: str):
    return hf_text(POEM_PROMPT.format(context=context), temperature=0.85, max_new_tokens=400)

def gen_image(context: str):
    img_prompt = hf_text(IMAGE_PROMPT_TMPL.format(context=context), temperature=0.6, max_new_tokens=120)
    url = f"https://api-inference.huggingface.co/models/{MODEL_IMAGE}"
    headers = {"Authorization": f"Bearer {HF_TOKEN}"}
    payload = {"inputs": img_prompt}
    r = requests.post(url, headers=headers, json=payload, timeout=180)
    r.raise_for_status()
    return r.content

def save_mp3(text: str):
    path = OUTPUT_DIR / f"tts_{uuid.uuid4().hex}.mp3"
    gTTS(text).save(str(path))
    return path

# ------------- UI -------------
st.set_page_config(page_title="DreamSense", page_icon="🌙", layout="centered")
st.title("🌙 DreamSense")
st.caption("Type your dream. We'll ask a few questions, then craft something beautiful.")

dream = st.text_area("📝 Describe your dream", height=200)

if dream and st.button("Get follow-up questions"):
    st.session_state.questions = gen_followups(dream)
    st.session_state.answers   = [""] * len(st.session_state.questions)

if "questions" in st.session_state:
    st.subheader("🔍 Follow-up Questions")
    for i, q in enumerate(st.session_state.questions):
        st.session_state.answers[i] = st.text_input(q, key=f"fq_{i}")

    output_type = st.radio("Output type", ["Poem / Prose (text)", "Image + Poem"], index=0)
    do_tts = st.checkbox("🔊 Read it aloud")

    if all(st.session_state.answers) and st.button("Generate ✨"):
        context = "Dream: " + dream + "\nAnswers:\n" + "\n".join(f"- {a}" for a in st.session_state.answers)

        if output_type.startswith("Image"):
            prose = gen_poem(context)
            img_bytes = gen_image(context)
            st.markdown("### 🖼️ Image")
            st.image(Image.open(BytesIO(img_bytes)))
            st.markdown("### 📝 Poetic Interpretation")
            st.write(prose)
        else:
            prose = gen_poem(context)
            st.markdown("### 📝 Poetic Interpretation")
            st.write(prose)

        if do_tts:
            mp3_path = save_mp3(prose)
            st.audio(str(mp3_path))


2025-07-25 12:04:38.969 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 12:04:38.971 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 12:04:38.971 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 12:04:38.972 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 12:04:38.972 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 12:04:38.973 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 12:04:38.975 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 12:04:38.978 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [163]:
# ONLY run if you want to preview the web app right from the notebook.
# Stop with the stop button in Jupyter or Ctrl+C in terminal.
import subprocess, sys, time

proc = subprocess.Popen([sys.executable, "-m", "streamlit", "run", "streamlit_app.py", "--server.headless", "true"])
time.sleep(3)
print("Streamlit is running. Copy the URL shown above or in the terminal output.")





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8520
  Network URL: http://192.168.0.32:8520
  External URL: http://92.233.134.66:8520

  For better performance, install the Watchdog module:

  $ xcode-select --install
  $ pip install watchdog
            
Streamlit is running. Copy the URL shown above or in the terminal output.
